In [1]:
!pip install nbimporter

In [ ]:
from scenario1 import resolve_missing_signs

In [16]:
import geopandas as gpd
import json
from geopy.distance import geodesic

# Load datasets
def load_data(base_path):
    return {
        'validations': gpd.read_file(f"{base_path}validations.geojson"),
        'signs': gpd.read_file(f"{base_path}signs.geojson"),
        'topology': gpd.read_file(f"{base_path}full_topology_data.geojson")
    }

datasets = load_data("data/")


In [17]:
# Helper functions

# Extract pedestrian access flag from a road row
def get_pedestrian_access(road):
    try:
        return json.loads(road.accessCharacteristics)[0].get("pedestrian", True)
    except (json.JSONDecodeError, IndexError, KeyError):
        return True

# Function to compute the nearest road for a given sign point
def find_nearest_road(sign_point, topology, target_crs='EPSG:3857'):
    # Convert sign_point to a GeoSeries using topology's CRS
    sign_series = gpd.GeoSeries([sign_point], crs=topology.crs)
    
    # Reproject both sign and topology to a projected CRS (meters)
    sign_projected = sign_series.to_crs(target_crs).iloc[0]
    topology_projected = topology.to_crs(target_crs)
    
    nearest = None
    min_dist = float('inf')
    
    for road in topology_projected.itertuples():
        dist = sign_projected.distance(road.geometry)
        # Optionally, print each distance (in meters)
        # print("Distance (m):", dist)
        if dist < min_dist and not get_pedestrian_access(road):
            min_dist, nearest = dist, road
    return nearest

def get_optimal_crs(gdf):
    """Auto-detect UTM zone from data centroid for meter-based accuracy"""
    try:
        centroid = gdf.geometry.unary_union.centroid
        lat, lon = centroid.y, centroid.x
        utm_zone = utm.from_latlon(lat, lon)
        epsg = 32600 + utm_zone[2] if lat >= 0 else 32700 + utm_zone[2]
        return f"EPSG:{epsg}"
    except:
        return "EPSG:3857"  # Web Mercator fallback

def update_topology_motorway_flag(topology_gdf, road_id):
    """Update motorway flag in topology dataset"""
    idx = topology_gdf[topology_gdf.id == road_id].index
    if not idx.empty:
        try:
            char = json.loads(topology_gdf.loc[idx[0], 'topologyCharacteristics'])
            char['isMotorway'][0]['value'] = True
            topology_gdf.loc[idx[0], 'topologyCharacteristics'] = json.dumps(char)
        except (KeyError, json.JSONDecodeError):
            pass
    return topology_gdf

In [ ]:
crs_code = get_optimal_crs(datasets['signs'])

In [18]:
def process_violations():
    for idx, violation in datasets['validations'].iterrows():
        print(f"Processing violation index: {idx}")
        sign_id = violation['Feature ID']
        sign_data = datasets['signs'][datasets['signs']['id'] == sign_id]
        
        if sign_data.empty:
            print(f"No sign found for violation with Feature ID {sign_id}. Skipping.\n")
            continue

        # Scenario 1: Check sign existence using satellite imagery
        missing_signs = resolve_missing_signs(datasets['validations'])
        is_sign = True
        if sign_id in missing_signs:
            is_sign = False
        
        if not is_sign:
            print(f"Scenario 1: Satellite imagery missing for sign {sign_id}. Removing sign.")
            datasets['signs'].drop(sign_data.index, inplace=True)
            continue
        else:
            print(f"Scenario 1: Sign {sign_id} verified in satellite imagery.")

        # Scenario 2/3: Find and validate nearest road
        nearest_road = find_nearest_road(sign_point, datasets['topology'])
        if nearest_road is not None:
            # The road geometry from nearest_road is in EPSG:3857 due to our projection in find_nearest_road.
            # We need to convert it back to geographic CRS (EPSG:4326) to calculate a valid geodesic distance.
            road_geom_projected = gpd.GeoSeries([nearest_road.geometry], crs='EPSG:3857')
            road_geom_geo = road_geom_projected.to_crs('EPSG:4326').iloc[0]
            road_centroid_geo = road_geom_geo.centroid
            
            # Get coordinates as (lat, lon) tuples
            sign_latlon = (float(sign_point_geo.y), float(sign_point_geo.x))
            centroid_latlon = (float(road_centroid_geo.y), float(road_centroid_geo.x))
            
            try:
                distance_m = geodesic(sign_latlon, centroid_latlon).meters
                print(f"Scenario 2/3: Distance between sign and road centroid: {distance_m:.2f} m")
            except Exception as e:
                print(f"Error computing geodesic distance: {e}")
                distance_m = float('inf')
            
            if distance_m < 20:
                print(f"Updating sign {sign_id}: Assigning topology ID {nearest_road.id} (within {distance_m:.2f} m).")

                access_idx = datasets['topology'][datasets['topology'].id == nearest_road.id].index
                if not access_idx.empty:
                    # Update pedestrian access
                    datasets['topology'].loc[access_idx, 'accessCharacteristics'] = \
                        json.dumps([{"pedestrian": False}])
                    
                    # Update topology motorway flag
                    datasets['topology'] = update_topology_motorway_flag(datasets['topology'], nearest_road.id)
                    print(f"Updated access characteristics for road {nearest_road.id} to pedestrian=False and isMotorway = true")
                else:
                    print(f"No access characteristics found for road {nearest_road.id}.")
            else:
                print(f"Sign {sign_id} is more than 20 m away from nearest road (distance: {distance_m:.2f} m).")
        else:
            print(f"Scenario 2/3: No valid nearest road found for sign {sign_id}.")

        # Scenario 4: Mark legitimate exceptions based on pedestrian access flag
        status = 'LEGITIMATE_EXCEPTION' if (
            nearest_road is not None and get_pedestrian_access(nearest_road) and distance_m < 20
        ) else 'No Violation'
        datasets['validations'].loc[idx, 'Status'] = status
        print(f"Scenario 4: Violation {idx} marked as {status}.\n")


Processing violation index: 0
Found sign urn:here::here:signs:1621752874458049432 at geographic location: (49.13894509, 8.16270644)
Tile saved successfully.
Scenario 1: Sign urn:here::here:signs:1621752874458049432 verified in satellite imagery.
Scenario 2/3: Distance between sign and road centroid: 33.48 m
Sign urn:here::here:signs:1621752874458049432 is more than 20 m away from nearest road (distance: 33.48 m).
Scenario 4: Violation 0 marked as No Violation.

Processing violation index: 1
Found sign urn:here::here:signs:1621752876050240343 at geographic location: (49.14067207, 8.16831829)
Tile saved successfully.
Scenario 1: Sign urn:here::here:signs:1621752876050240343 verified in satellite imagery.
Scenario 2/3: Distance between sign and road centroid: 55.66 m
Sign urn:here::here:signs:1621752876050240343 is more than 20 m away from nearest road (distance: 55.66 m).
Scenario 4: Violation 1 marked as No Violation.

Processing violation index: 2
Found sign urn:here::here:signs:162175

In [ ]:
# Execute the process
process_violations()

# Save updated datasets
datasets['signs'].to_file("corrected_signs.geojson", driver='GeoJSON')
datasets['signs'].to_file("corrected_topology_access_chars.geojson", driver='GeoJSON')
datasets['validations'].to_file("corrected_validations.geojson", driver='GeoJSON')

## Visualization for Scenario 2

In [23]:
import folium

# Sign point coordinates (latitude, longitude)
sign_lat, sign_lon = 49.15669894, 8.1538361

# Create a Folium map centered on the sign point
m = folium.Map(location=[sign_lat, sign_lon], zoom_start=15)

# Add a marker for the sign point
folium.Marker(
    location=[sign_lat, sign_lon],
    popup="Sign Point",
    icon=folium.Icon(color="red", icon="info-sign")
).add_to(m)

# Blue line string coordinates provided in (lon, lat) order:
blue_line_coords_lonlat = [
    [8.15485, 49.16135],
    [8.15498, 49.16081],
    [8.15505, 49.15978],
    [8.1550328, 49.1592224],
    [8.15502, 49.15881],
    [8.15498, 49.15853],
    [8.15487, 49.15817],
    [8.1548, 49.1580074],
    [8.1547328, 49.1578514],
    [8.1546616, 49.1576862],
    [8.15459, 49.15752],
    [8.1545184, 49.1573053],
    [8.15443, 49.15704],
    [8.15439, 49.15695],
    [8.15418, 49.15688],
    [8.154, 49.1568],
    [8.15397, 49.15674]
]

# Convert blue line coordinates to (lat, lon) order for Folium:
blue_line_coords = [[lat, lon] for lon, lat in blue_line_coords_lonlat]

# Add the blue line string as a polyline to the map
folium.PolyLine(
    locations=blue_line_coords,
    color="blue",
    weight=5,
    opacity=0.7,
    popup="Corrected Road (Blue)"
).add_to(m)

# Define red line string coordinates (lon, lat order). 
# Replace these with your actual red line string coordinates.
red_line_coords_lonlat = [
    [8.1545, 49.1575],
    [8.1544, 49.1570],
    [8.1543, 49.1565],
    [8.1542, 49.1560]
]

# Convert red line coordinates to (lat, lon) order:
red_line_coords = [[lat, lon] for lon, lat in red_line_coords_lonlat]

# Add the red line string as a polyline to the map
folium.PolyLine(
    locations=red_line_coords,
    color="red",
    weight=5,
    opacity=0.7,
    popup="Additional Road (Red)"
).add_to(m)

# Display the map in the notebook
m
